In [14]:
import pandas as pd
# File serves to read data from the purchases query and clean it. 

In [16]:
df_p23 = pd.read_csv("FY23 Purchases.csv")
df_p24 = pd.read_csv("FY24 Purchases.csv")

In [18]:
df_p23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81481 entries, 0 to 81480
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ComputerID    69211 non-null  object 
 1   ComputerName  69211 non-null  object 
 2   PurchaseDate  81481 non-null  object 
 3   Email         61980 non-null  object 
 4   FIRST_NAME    62046 non-null  object 
 5   LAST_NAME     62044 non-null  object 
 6   ID_NUMBER     62046 non-null  object 
 7   PRODUCT_NAME  81481 non-null  object 
 8   QUANTITY      81481 non-null  int64  
 9   UNIT_PRICE    81481 non-null  float64
 10  ID            81481 non-null  object 
 11  PRODUCT_ID    81481 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.5+ MB


In [20]:
# Finds the index of all the entries where the quantity is negative (i.e. a refund)
# then matches that product ID and email (since some entries do no have ID_NUMBER)
# then drops both the refund row and the purchase that was associated with the refund
def drop_pairs(df):
    negatives = df["QUANTITY"] < 0
    refund_indexes = df[negatives].index
    indexes_to_drop = set(refund_indexes)

    for index in refund_indexes:
        product_id = df.at[index, "PRODUCT_ID"]
        email = df.at[index, "Email"]
        matching_purchase = df[(df["PRODUCT_ID"] == product_id) & 
                               (df["Email"] == email) & 
                               (df["QUANTITY"] > 0)]
        
        # If a matching purchase exists, add its index to the drop set
        if not matching_purchase.empty:
            purchase_index = matching_purchase.index[0]
            indexes_to_drop.add(purchase_index)
    df_dropped = df.drop(indexes_to_drop)
    return df_dropped


df1 = drop_pairs(df_p23)
df2 = drop_pairs(df_p24)


In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77745 entries, 0 to 81480
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ComputerID    66185 non-null  object 
 1   ComputerName  66185 non-null  object 
 2   PurchaseDate  77745 non-null  object 
 3   Email         58323 non-null  object 
 4   FIRST_NAME    58388 non-null  object 
 5   LAST_NAME     58386 non-null  object 
 6   ID_NUMBER     58388 non-null  object 
 7   PRODUCT_NAME  77745 non-null  object 
 8   QUANTITY      77745 non-null  int64  
 9   UNIT_PRICE    77745 non-null  float64
 10  ID            77745 non-null  object 
 11  PRODUCT_ID    77745 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.7+ MB


In [24]:
refunds=(df_p23["QUANTITY"]<0).sum()
print(refunds) # number of refunds in the dataframe


2305


In [26]:
total_removed = len(df_p23)-len(df1)
total_removed #number of removed refunds and their respective purchases

3736

In [28]:
missed_transactions = (refunds*2)-total_removed 
missed_transactions              

# calculating the number of transactions that did not 
# have their purchase pair removed

# these transactions are missed because the purchase 
# happened before the specified query date

#DROP ALL THE REFUNDS ANYWAYS. WAS NOT SOLD IN THIS TIME PERIOD.

874

In [30]:
#A check to of the function
total_online_purchases = len(df1)-df1["ComputerID"].count()
total_online_purchases
total_inperson_purchases = len(df1)-total_online_purchases
print(total_inperson_purchases, "In-person purchases\n", total_online_purchases, "Online Purchases")

66185 In-person purchases
 11560 Online Purchases


In [34]:
# saves only the required columns needed 
df1=df1[['ComputerName', 'PurchaseDate', 'ID_NUMBER','PRODUCT_NAME', 'QUANTITY', 'UNIT_PRICE']]
df2=df2[['ComputerName', 'PurchaseDate', 'ID_NUMBER','PRODUCT_NAME', 'QUANTITY', 'UNIT_PRICE']]
# FIlls in empty computer names with 'Online Purchase' since they were conducted online. 
df1['ComputerName'] = df1['ComputerName'].fillna("Online Purchase")
df2['ComputerName'] = df2['ComputerName'].fillna("Online Purchase")


In [38]:
df1.to_csv("Filtered_Cleaned FY23 Purchases.csv", index=False)
df2.to_csv("Filtered_Cleaned FY24 Purchases.csv", index=False)